In [11]:
import numpy as np
import os
from tqdm import tqdm
import requests
import matplotlib.pyplot as plt
import json

In [ ]:
#resp = requests.get('https://aydanomachado.com/mlclass/02_Optimization.php?phi1={}&theta1={}&phi2={}&theta2={}&phi3={}&theta3={}&dev_key=Andre%20Santos'.format(chromosome[0],chromosome[1],chromosome[2],chromosome[3],chromosome[4],chromosome[5]))
#resp = requests.get('http://localhost:8080/antenna/simulate?phi1={}&theta1={}&phi2={}&theta2={}&phi3={}&theta3={}'.format(chromosome[0],chromosome[1],chromosome[2],chromosome[3],chromosome[4],chromosome[5]))

In [8]:
'''
Funcão utilizada para computar o valor de ajuste de cada solução da atual
população. Retorna a lista de [chomossomo,valor_de_ajuste] e prob_fits que
contém a probabilidade de cada cromossomo para a fase de seleção
'''

def fitness_function(population):
    evaluated = []
    for chromosome in population:
        resp = requests.get('https://aydanomachado.com/mlclass/02_Optimization.php?phi1={}&theta1={}&phi2={}&theta2={}&phi3={}&theta3={}&dev_key=Andre%20Santos'.format(chromosome[0],chromosome[1],chromosome[2],chromosome[3],chromosome[4],chromosome[5]))
        resp = resp.text
        resp = resp.replace("'", "\"")
        resp = json.loads(resp)
        resp = resp['gain']
        evaluated.append([chromosome,resp])
    evaluated.sort(key=lambda x:x[1], reverse=True)
    prob_fits = [x[1] for x in evaluated]
    prob_fits = np.exp(prob_fits) / np.sum(np.exp(prob_fits))
    return evaluated,prob_fits

In [15]:
'''
Função que faz o cruzamento de dois cromossomos dados, retorna dois cromossomos filhos
'''
def crossover(father1,father2):

    #child1 = [np.hstack((father1[0][:2],father2[0][2:4],father1[0][4:])),0.0]
    #child2 = [np.hstack((father2[0][:2],father1[0][2:4],father2[0][4:])),0.0]
    
    child1 = [np.hstack((father1[0][:2],father2[0][2:4],father1[0][4:])),0.0]
    child2 = [np.hstack((father2[0][:2],father1[0][2:4],father2[0][4:])),0.0]

    return child1,child2

In [16]:
'''
Função que aplicaa mutação a um cromossomo dado, a probabilidade de mutação para um 
determinado cromossomo é de 5%
'''
def mutation(new_generation):

    prob_mutation = 10
    muted = []
    for chrom in new_generation:
        if np.random.randint(0,100) <= prob_mutation:
            position = np.random.randint(0,5)
            chrom[0][position] = np.random.randint(0,359)
            muted.append(chrom[0])
        else:
            muted.append(chrom[0])
    return muted

In [5]:
def selection(ordened_population,prob_fits, rate_sub=0.6):
    def select_father(ordened_population, prob_fits):
        
        choice = np.random.uniform(0,1)
        count_sum = 0
        for i,chrom in zip(range(len(prob_fits)),prob_fits):        
            count_sum += chrom
            if count_sum >= choice:
                return ordened_population[i]
    
    qtd_remain = int(np.floor(len(ordened_population)*(1 - rate_sub))) #quantidade de cromossomoes que sobreviverão
    qtd_sub = int(np.ceil(len(ordened_population)*rate_sub)) #quantidade de cromossomos que serão substituídos
    qtd_crossover = qtd_sub // 2 #quantidade de cruzamento entre 2 pais (que produz 2 filhos)
    next_generation = ordened_population[:qtd_remain] #adicionando os sobreviventes a proxima geração
   
    for i in range(qtd_crossover): 
        father1 = select_father(ordened_population[::-1],prob_fits[::-1])
        father2 = None
        
        while(True):
            father2 = select_father(ordened_population[::-1],prob_fits[::-1])
            if not (father2[0] == father1[0]).all():
                break
                
        child1,child2 = crossover(father1,father2)
        #print('PAI_1 = {} e PAI_2= {} - > FILHO_1 = {} e FILHO_2= {}'.format(father1[0],father2[0],child1[0],child2[0]))
        next_generation.append(child1) 
        next_generation.append(child2)
        
    return next_generation

In [9]:
def genetic_algorithm(population_init,n_gen,rate_sub=0.6):
    
    current_gen = population_init
    best_config = None
    best_value_fits = -float('inf')
    count = 0
    
    for gen in tqdm(range(n_gen)):
        current_gen,prob_current_gen = fitness_function(current_gen)
        
        if best_value_fits < current_gen[0][1]:
            best_config = current_gen[0][0]
            best_value_fits = current_gen[0][1]
        else:
            count +=1
        
        #print(best_value_fits,current_gen[0][1])
        '''
        if count == 3:
            print('5 gerações em melhora')
            print('geração ={}'.format(gen))
            return best_config,best_value_fits
        '''
        selected = selection(current_gen,prob_current_gen,rate_sub)
        muted = mutation(selected)
        
        next_gen = muted
        current_gen = next_gen
    
    return best_config,best_value_fits 

In [17]:
init_pop = np.random.randint(0,359,(4,6))
n_gen = 15
best_config, best_value = genetic_algorithm(init_pop, n_gen,0.6)
print('Melhor configuração = {}, melhor valor = {}'.format(best_config, best_value))

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [02:28<00:00,  9.89s/it]

Melhor configuração = [321  26 101 139 249  46], melhor valor = 2.0090328643
